In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
dbutils.widgets.text("catalogo", "catalog_jptq")
dbutils.widgets.text("esquema_source", "bronze_jptq")
dbutils.widgets.text("esquema_stage", "silver_jptq")
dbutils.widgets.text("storageLocation", "abfss://unity-catalog-jptq@adlsjptq0126.dfs.core.windows.net")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_stage = dbutils.widgets.get("esquema_stage")
storageLocation = dbutils.widgets.get("storageLocation")

In [0]:
def fn_transform_director_pelicula(catalogo: str, esquema_source: str):
   sql_select_director = f"""SELECT
                              DISTINCT
                              COD_DIRECTOR,
                              DES_DIRECTOR
                              FROM ( SELECT
                                     CONCAT(
                                     SUBSTRING(SPLIT(DES_DIRECTOR,' ')[0],1,3),
                                     SUBSTRING(ELEMENT_AT(SPLIT(DES_DIRECTOR,' '),-1),1,3)
                                     ) AS COD_DIRECTOR,
                                     DES_DIRECTOR
                                     FROM {catalogo}.{esquema_source}.TBL_DETALLE_PELICULAS
                                     GROUP BY DES_DIRECTOR
                                     ORDER BY DES_DIRECTOR ASC)"""
   df_t_director_pelicula = spark.sql(sql_select_director)
   return df_t_director_pelicula

In [0]:
df_tmp_director_pelicula = fn_transform_director_pelicula(catalogo, esquema_source)

In [0]:
df_tmp_director_pelicula = df_tmp_director_pelicula.withColumn("FEC_CARGA", current_timestamp())

In [0]:
display(df_tmp_director_pelicula)

In [0]:
#df_tmp_director_pelicula.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_stage}.TBL_TMP_DIRECTOR_PELICULA")
df_tmp_director_pelicula.createOrReplaceTempView("tmp_df_tmp_director_pelicula")

In [0]:
def fn_create_table_tmp_director_pelicula(catalogo: str, esquema_stage: str):
   if spark.catalog.tableExists(f"{catalogo}.{esquema_stage}.TBL_TMP_DIRECTOR_PELICULA"):
      print(f"La tabla ya existe: {catalogo}.{esquema_stage}.TBL_TMP_DIRECTOR_PELICULA")
   else:
      spark.sql(f"""CREATE TABLE IF NOT EXISTS {catalogo}.{esquema_stage}.TBL_TMP_DIRECTOR_PELICULA (
        COD_DIRECTOR STRING,
        DES_DIRECTOR STRING,
        FEC_CARGA TIMESTAMP
        )
      USING DELTA
      LOCATION '{storageLocation}/silver-jptq/TBL_TMP_DIRECTOR_PELICULA'""")
      print(f"Tabla creada correctamente: {catalogo}.{esquema_stage}.TBL_TMP_DIRECTOR_PELICULA")

In [0]:
fn_create_table_tmp_director_pelicula(catalogo, esquema_stage)

In [0]:
def fn_truncate_table_tmp_director_pelicula(catalogo: str, esquema_stage: str):
    spark.sql(f"""
    TRUNCATE TABLE {catalogo}.{esquema_stage}.TBL_TMP_DIRECTOR_PELICULA
    """) 

In [0]:
fn_truncate_table_tmp_director_pelicula(catalogo, esquema_stage)

In [0]:
def fn_insert_table_tmp_director_pelicula(catalogo: str, esquema_stage: str):
  spark.sql(f"""
  INSERT INTO {catalogo}.{esquema_stage}.TBL_TMP_DIRECTOR_PELICULA
  SELECT * FROM tmp_df_tmp_director_pelicula
  """) 

In [0]:
fn_insert_table_tmp_director_pelicula(catalogo, esquema_stage) 

In [0]:
%sql
SELECT
*
FROM
catalog_jptq.silver_jptq.tbl_tmp_director_pelicula